In [63]:
import numpy as np
import pandas as pd

In [64]:
!ls ../input/titanic

gender_submission.csv  test.csv  train.csv


In [65]:
train = pd.read_csv("../input/titanic/train.csv")
test = pd.read_csv("../input/titanic/test.csv")
gender_submission = pd.read_csv("../input/titanic/gender_submission.csv")

In [66]:
gender_submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [67]:
data = pd.concat([train, test], sort=False)

In [68]:
print(len(train), len(test), len(data))

891 418 1309


In [69]:
data.isnull().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

In [70]:
data['Sex'].replace(['male','female'], [0,1], inplace=True)

In [71]:
data['Embarked'].fillna(('S'), inplace=True)
data['Embarked'] = data['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

In [72]:
data['Fare'].fillna(np.mean(data['Fare']), inplace=True)
data['fare_value']=data['Fare']/50

In [73]:
age_avg = data['Age'].mean()
age_std = data['Age'].std()
data['Age'].fillna(np.random.randint(age_avg - age_std, age_avg + age_std), inplace=True)
data['age_value']=data['Age']/50

In [74]:
data['family'] = (data['SibSp']+data['Parch'])/5 

In [75]:
data['isAlone'] = 0
data.loc[data['family'] > 0, 'isAlone'] = 1

In [76]:
delete_columns = ['Name','PassengerId','SibSp','Parch','Ticket','Cabin','Age','Fare']
data.drop(delete_columns, axis=1, inplace=True)

In [77]:
train = data[:len(train)]
test = data[len(train):]

In [78]:
y_train0 = train['Survived']
X_train0 = train.drop('Survived', axis = 1)
X_test0 = test.drop('Survived', axis = 1)

In [79]:
X = np.array(X_train0)
y = np.array(y_train0)

In [80]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors = 13)

In [81]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

k_fold = KFold(n_splits=20, shuffle=True, random_state=0)

for train_index, test_index in k_fold.split(X):

    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = y[train_index], y[test_index]

    clf.fit(X_train, Y_train) 
    
    scoring = 'accuracy'
    score = cross_val_score(clf, X_test, Y_test, cv=k_fold, n_jobs=1, scoring=scoring)
    print(score)

[0.77777778 0.77777778 0.33333333 1.         0.77777778 0.66666667
 0.77777778 0.77777778 0.88888889 0.55555556]
[1.         0.66666667 0.77777778 0.77777778 0.77777778 0.77777778
 0.55555556 0.88888889 0.66666667 0.625     ]
[0.55555556 0.88888889 0.88888889 0.44444444 0.77777778 0.55555556
 0.55555556 0.88888889 0.77777778 0.875     ]
[0.77777778 0.77777778 0.88888889 0.88888889 0.66666667 0.77777778
 0.55555556 0.55555556 0.55555556 0.625     ]
[0.77777778 0.88888889 0.77777778 0.77777778 0.88888889 0.77777778
 0.66666667 0.77777778 0.77777778 0.75      ]
[0.77777778 0.77777778 0.55555556 0.77777778 1.         1.
 0.66666667 0.77777778 0.66666667 0.875     ]
[0.88888889 0.77777778 0.88888889 1.         1.         0.77777778
 0.66666667 0.44444444 0.77777778 0.5       ]
[0.44444444 0.77777778 0.55555556 0.88888889 0.77777778 0.88888889
 0.77777778 0.88888889 0.66666667 0.625     ]
[0.66666667 0.77777778 0.88888889 0.66666667 0.55555556 0.77777778
 0.77777778 0.88888889 0.88888889 0.7

In [82]:
y_pred = clf.predict(np.array(X_test0))
y_pred = y_pred.astype(np.int)

In [83]:
y_pred[:20]

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0])

In [84]:
sub = gender_submission
sub['Survived'] = list(map(int, y_pred))
sub.to_csv("submission.csv", index=False)

In [85]:
sub

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
